# Process Mining с sberpm

Библиотека __`sberpm`__ предназначена для анализа и исследования процессов, построения их в виде графов и решения смежных задач классификации и кластеризации с использованием алгоритмов машинного обучения.

## Оглавление

[DataHolder](#DataHolder)

[Синтетические ID процесса](#Синтетические-ID-процесса)

[----------  Традиционный-Process-Mining  ----------](#----------Традиционный-Process-Mining----------)

I. [Майнеры и визуализация графов](#I.-Майнеры-и-визуализация-графов)
 1. [SimpleMiner](#1.-SimpleMiner)
 2. [CausalMiner](#2.-CausalMiner)
 3. [HeuMiner](#3.-HeuMiner)
 4. [AlphaMiner](#4.-AlphaMiner)
 5. [AlphaPlusMiner](#5.-AlphaPlusMiner)
 6. [InductiveMiner](#6.-InductiveMiner)

II. [Метрики](#II.-Метрики)
- [Метрики + графы](#Метрики-+-графы)

III. [Conformance Checking](#III.-Conformance-Checking)

 IV. [BPMN](#IV.-BPMN)

V. [Визуализация](#V.-Визуализация)



[---------- Машинное-обучение ----------](#----------Машинное-обучение----------)

[Проверка наличия моделей](#Проверка-наличия-моделей)

I. [Кластеризация этапов](#I.-Кластеризация-этапов)

II. [Автоматический поиск неэффективностей](#II.-Автоматический-поиск-неэффективностей)

III. [Поиск аномалий](#III.-Поиск-аномалий)

IV. [Факторный анализ](#IV.-Факторный-анализ)

V. [Рекомендательная система](#V.-Рекомендательная-система)

VI. [Анализ текстов](#VI.-Анализ-текстов)

VII. [Сентиментный анализ](#VII.-Сентиментный-анализ)

VIII. [Поиск счастливого пути](#VIII.-Поиск-счастливого-пути)

IX. [Предсказание структуры графа](#IX.-Предсказание-структуры-графа)

X. [Имитационное моделирование и what-if анализ](#X.-Имитационное-моделирование-и-what-if-анализ)

XI. [Decision Mining](#XI.-Decision-Mining)

XII. [Хронометраж](#XII.-Хронометраж)


# DataHolder

`DataHolder` – это базовый класс для хранения данных. Практически все алгоритмы библиотеки работают с ним (принимают на вход).

Для создания класса `DataHolder` необходимо сперва указать путь к файлу или передать DataFrame конструктору, а затем указать __id_column__ и __activity_column__. Однако, для большинства алгоритмов Process Mining, представленных в библиотеке, этих столбцов недостаточно – необходимы хотя бы одна колонка времени (__start_timestamp_column__ и/или __end_timestamp_column__) и колонка пользователей (__user_column__). 

## Параметры DataHolder
- **data (str or pd.DataFrame)** – путь к файлу данных (.csv, .xls(x), .txt) или pd.DataFrame
- **id_column (str)** – столбец id
- **activity_column (str)** – столбец активностей
- __<font color='red'>*</font>start_timestamp_column (str)__ – время начала активностей
- __<font color='red'>*</font>end_timestamp_column (str)__ – время окончания активностей
- __user_column (str)__ – столбец с именами/id пользователей
- __text_column (str)__ – столбец с текстовыми данными
- __duration_column (str)__ – столбец с длительностями активностей (если не задается, то расчитывается как время_активности_2 - время_активности_1, причем если есть только один столбец со временем, то для последней активности в цепочке ставится NaN)
- __duration_unit (str)__ – размерность (единица измерения) значений в столбце duration_column, если он задан

- __sep (str, default=',')__ – разделительный знак (используется только при чтении данных из файла)
- __encoding (str)__ – кодировка (используется только при чтении данных из файла)
- __nrows (int)__ – количество строк для чтения (используется только при чтении данных из файла)

- __preprocess (bool, default=True)__ – предобработка данных (сортировка, удаление None-значений, преобразование типов)
- __time_format (str)__ – формат временных колонок (обязательно задавать для правильного распознавания даты и ускорения работы). Правила написания: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
- __time_errors: (str, default='raise')__ – действие при ошибке конвертации
- __dayfirst: (bool, default=None)__ – True, если день стоит в начале строки
- __yearfirst: (bool, default=None)__ – True, если год стоит в начале строки
- __n_jobs (int, default=1)__ – максимальное количество потоков, доступное для некоторых вычислений


__<font color='red'>*</font>__ Для большинства алгоритмов нужно задать хотя бы один из временных столбцов. Если нет информации о типе столбца (время начала или конца), следует задать его как __start_timestamp_column__. Для верного распознавания формата также необходимо указать __time_format__.

Исследуемые данные должны представлять собой журнал событий (лог-файл), в котором хранится информация о последовательности (цепочке) событий (активностей) в бизнес-процессах. Пример журнала событий: $W = \{(a,b,c,d), (a,c,b,d), (a,e,d)\}$, где события $a$, $b$, $c$, $d$ и $e$ сортируются по времени.

## Создание DataHolder 
### – с помощью DataFrame

In [ ]:
from sberpm import DataHolder
import pandas as pd

df = pd.DataFrame({'id_column': [1, 1, 2, 2, 3, 3],
                   'activity_column': ['st1', 'st2', 'st1', 'st3', 'st1','st2'],
                   'start_timestamp_column': ['10.05.2020', '10.09.2020', '10.03.2020', '10.04.2020', '10.05.2020', '10.05.2020']})

data_holder = DataHolder(data=df, 
                         id_column='id_column', 
                         activity_column='activity_column', 
                         start_timestamp_column='start_timestamp_column', 
                         time_format='%d.%m.%Y')

### – с помощью указания пути файла

In [ ]:
path = 'example.xlsx'
data_holder = DataHolder(data=path, 
                         id_column='id', 
                         activity_column='stages', 
                         start_timestamp_column='dt', 
                         user_column='users', 
                         text_column="some_text",
                         time_format='%Y-%m-%d')

Если данные имеют какой-нибудь разделитель, например '|' как в csv, то после задания колонок, нужно задать параметр __sep='|'__.

## Атрибуты DataHolder
В `DataHolder` названия столбцов хранятся в соответствующих переменных (т.е. нет необходимости запоминать названия колонок):
- id_column
- activity_column
- start_timestamp_column
- end_timestamp_column
- user_column
- text_column
- duration_column

Кроме того, в `DataHolder` хранятся исходные и сгруппированные данные в виде DataFrame, к которым можно обратиться следующим образом:
- data
- grouped_data

## Методы DataHolder
- __check_or_calc_duration__ – рассчитывает длительность каждой активности (в секундах), если это необходимо 
- __get_grouped_data__ – выводит сгруппированные данные по id и указанным колонкам (например, по activity_column и start_timestamp_column)
- __get_unique_activities__ – выводит список уникальных активностей
- __get_columns__ – выводит список с названиями колонок 
- __get_text__ – выводит колонку с текстом, если такая есть
- __get_timestamp_col__ – выводит временную колонку; если их имеется 2, то выводит start_time_column
- __is_interval__ – возвращает True, если это "интервальный лог" (у которого имеются обе временные колонки: начала и конца активности)
- __top_traces_dh__ – возвращает data_holder с данными для n самых частых цепочек

In [ ]:
data_holder.check_or_calc_duration()

In [ ]:
data_holder.data.head()

In [ ]:
data_holder.get_grouped_data(data_holder.activity_column, data_holder.start_timestamp_column).head()

In [ ]:
dh_3 = data_holder.top_traces_dh(3)  # данные только для топ 3 цепочек
dfg = dh_3.get_grouped_data(dh_3.activity_column)
dfg.value_counts(dh_3.activity_column)  # проверка

Имея данные о бизнес-процессе с цепочками статусов и временем начала каждого из них, можно загрузить их в `DataHolder` и построить граф, максимально описывающий этот бизнес-процесс.

# Синтетические ID процесса

__Pro_n_check__ нумерует экземпляры процессов в зависимости от заданных условий. Создает столбец 'pro_n'.

Параметры:
- __data__ - объект хранения данных в формате DataFrame.
- __proc_columns__ - список столбцов с этапами процесса.
- __identifier_cols__ - список столбцов с идентификаторами процессов (если один из идентификаторов, указанных в списке, изменяется, считается, что запущен новый процесс).
- __start_proc__ - список строковых значений, указывающих на начало процесса.
- __end_proc__ - список строковых значений, указывающих на завершение процесса.
- __sort_params__ - список столбцов для сортировки (должен использоваться в случае смешанных данных).

Методы:
- __get_pro_n()__ - при запуске добавляется столбец 'pro_n' к исходному датасету. (Возвращает DataFrame)
- __get_result()__ - получить готовый результат. Работает, если метод 'get_pro_n' был запущен ранее. (Возвращает DataFrame)

In [ ]:
from sberpm.pro_n_check import Pro_n_check

In [ ]:
df = pd.read_excel('example.xlsx')
pro_n = Pro_n_check(data = df,
                    proc_columns= ['stages'],
                    identifier_cols = ['id'],
                    start_proc = ['Stage_0'],
                    end_proc = ['Stage_2'],
                    sort_params = ['id','dt'])

In [ ]:
pro_n.get_pro_n()

# ----------Традиционный Process Mining----------

## I. Майнеры и визуализация графов

Для построения и отрисовки графа процесса в библиотеке реализовано несколько алгоритмов. Все они хранятся в модуле __`sberpm.miners`__ и имеют один метод:
- __apply__ – строит граф, который сохраняется в поле graph

### 1. SimpleMiner

`SimpleMiner` отрисовывает все ребра, найденные в логе (без какой-либо фильтрации).

В терминах Process Mining:
> Если хотя бы в одной цепочке активностей из лога за некоторой активностью $X$ непосредственно следует активность $Y$ (цепочка вида $...XY...$), то пишут $X>Y$ ($Y$ follows $X$, _follows_ relation).

SimpleMiner рисует ребра между такими парами активностей $X$ и $Y$, если выполняется $X>Y$.

In [ ]:
from sberpm.miners import SimpleMiner

In [ ]:
# Создание объекта SimpleMiner. В конструктор подается DataHolder и параметры алгоритма 
# (у данного майнера параметров нет)
simple_miner = SimpleMiner(data_holder)

# Запуск алгоритма построения графа
simple_miner.apply()

# Сохранение графа
graph = simple_miner.graph

### Визуализация графа
Для визуализации графа следует использовать `GraphvizPainter` из модуля __`sberpm.visual`__

In [ ]:
%matplotlib inline
from sberpm.visual import GraphvizPainter
import os
os.environ['PATH'] += os.pathsep + "C:/Program Files (x86)/Graphviz2.38/bin"

Класс `GraphvizPainter` имеет методы:
- __apply__ – принимает на вход граф, полученный с помощью майнера, и производит расчет для его отрисовки 
- __write_graph__ – сохраняет граф в требуемом формате (pdf, svg, gv, png)
- __show__ – выводит граф в notebook

In [ ]:
# Создание объекта GraphvizPainter
painter = GraphvizPainter()

# Расчет графа по результатам работы SimpleMiner
painter.apply(graph)

# Можно сохранить граф на жесткий диск в формате png, svg, pdf или gv
painter.write_graph('SimpleMiner.png', format='png')

# Можно вывыести граф в notebook
painter.show()

Класс `Graph` из модуля __`sberpm.visual`__ имеет методы:
- __get_nodes__ – получить все узлы
- __get_edges__ – получить все ребра
- __add_node_metric__ – добавить метрику, связанную с узлами графа
- __add_edge_metric__ – добавить метрику, связанную с ребрами графа
- __clear_node_metrics__ – удалить все метрики с нод (узлов)
- __clear_edge_metrics__ – удалить все метрики с ребер

### 2. CausalMiner

`CausalMiner` основан на фильтрации ребер.
> Производные типы связей от $X>Y$:
- прямые связи ($X \to Y$, _causal_ relation) – это связи, где $Х>Y$ и не $Y>X$
- параллельные связи($X\parallel Y $, _parallel_ relation) – это связи, где $Х>Y$ и $Y>X$
- независимые связи ($X\#Y$, independent) – это связи, где не $X>Y$ и не $Y>X$

CausalMiner рисует ребра между такими парами активностей $X$ и $Y$, если выполняется $X\to Y$.

In [ ]:
from sberpm.miners import CausalMiner

In [ ]:
# Майнер
causal_miner = CausalMiner(data_holder)
causal_miner.apply()
graph = causal_miner.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 3. HeuMiner

`HeuMiner` – это эврестический майнер, который удаляет наиболее редкие связи в зависимости от задаваемого порога (threshold). 

Параметр **threshold** принимает значения **от 0 до 1**. Чем он больше, тем меньше ребер на графе (оставшиеся ребра считаются более важными).

Источник: https://www.researchgate.net/publication/229124308_Process_Mining_with_the_Heuristics_Miner-algorithm

In [ ]:
from sberpm.miners import HeuMiner

In [ ]:
# Майнер
heu_miner = HeuMiner(data_holder, threshold=0.8)
heu_miner.apply()
graph = heu_miner.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 4. AlphaMiner

`AlphaMiner` рисует граф в виде сетей Петри с учетом прямых, параллельных и независимых связей между активностями. 

In [ ]:
from sberpm.miners import AlphaMiner

In [ ]:
# Майнер
alpha_miner = AlphaMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 5. AlphaPlusMiner

`AlphaPlusMiner` – имплементация Alpha+ майнера, который также рисует граф в виде сетей Петри с учетом связей, но в отличие от AlphaMiner может работать с одноцикловыми (one-loop) цепочками вида activity_1$\to$activity_1 (самоцикл).

In [ ]:
from sberpm.miners import AlphaPlusMiner

In [ ]:
# Майнер
alpha_miner_plus = AlphaPlusMiner(data_holder)
alpha_miner_plus.apply()
graph = alpha_miner_plus.graph

# Отрисовка
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

### 6. InductiveMiner

`InductiveMiner` создаёт дерево процесса. Лисья дерева - реальные активности процесса, остальные вершины - операторы. Есть 4 типа операторов: 
- ПОСЛЕДОВАТЕЛЬНЫЙ (`->`), 
- ИСКЛЮЧАЮЩЕЕ ИЛИ (`X`), 
- ПАРАЛЛЕЛЬНЫЙ (`||`), 
- ЦИКЛ (`*`).

Есть дополнительный 'оператор', который говорит о том, что было невозможно найти ни один из 4 операторов, представленных выше:
- СМЕШЕННАЯ МОДЕЛЬ ('`?`')

*Замечание*: некоторые из листьев дерева могут быть *скрытыми активностями*, отображаемыми чёрными прямоугольниками. Они не являются реальными активностями и используются только для сохранения правильной структуры дерева. 

Например, из лога, состоящего из двух цепочек процесса $W = \{(a, b, c), (a, c)\}$, можно получить следующее дерево процеса:        
`->(a, X(b, скрытая_активность), c)`.

Если во время очередной итерации алгоритм не может найти разрез графа (=подобрать один из 4 операторов), возможно добавить следубщее поведение: если существует активность А, при удалении которой удаётся подобрать оператор, алгоритм возвращает следующее дерево:            
`||(X(активность_А, скрытая_активность), граф_без_активности_А)` - то есть активность А считается параллельной остальному графу.


Это поведение может быть включено или выключено параметром **parallel_activity** в классе `InductiveMiner`.

In [ ]:
from sberpm.miners import InductiveMiner

In [ ]:
# Miner
inductive_miner = InductiveMiner(data_holder)
inductive_miner.apply()
graph = inductive_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## II. Метрики

На данный момент в модуле __`sberpm.metrics`__ есть 5 основных типов метрик:
1. `ActivityMetric` – метрики по активностям (группировка по activity_column)
2. `TransitionMetric` – метрики по переходам (группировка по уникальным переходам)
3. `IdMetric`– метрики по id (группировка по id_column)
4. `TraceMetric` – метрики по цепочкам активностей (группировка по уникальным цепочкам)
5. `UserMetric` – метрики по пользователям (группировка по user_column)

In [ ]:
from sberpm.metrics import ActivityMetric, TransitionMetric, IdMetric, TraceMetric, UserMetric

Параметры:
- __data_holder__ – объект типа DataHolder, для которого надо рассчитать метрики
- __time_unit__ – единица времени, по умолчанию расчет временных метрик происходит в часах
- __round__ – количество цифр после запятой (только для метрик, значения которых могут быть с плавающей точкой)

Общие методы для всех классов:
- __apply__ – расчет всех характеристик
- __calc_metrics(...)__ – расчет указанных метрик (соответствуют методам/названиям колонок в DataFrame из apply)
- __calculate_time_metrics__ – расчет временных характеристик

- __total_duration__ – расчет суммарного времени работы
- __min_duration__ – расчет минимального времени работы
- __max_duration__ – расчет максимального времени работы
- __mean_duration__ – расчет среднего времени работы
- __median_duration__ – расчет медианного времени работы
- __std_duration__ – расчет стандартного отклонения времени работы
- __var_duration__ – расчет дисперсии времени работы

Дополнительные методы:
- ActivityMetric
    - __count__ - сколько раз активность встречается в логе
    - __unique_ids__ - уникальные id для каждой активности
    - __unique_ids_num__ - количество уникальных id для каждой активности
    - __aver_count_in_trace__ - среднее количество раз встречаемости активности в цепочке
    - __loop_percent__ - процент зацикленности
    - __throughput__ - частота - количество выполненных активностей за единицу времени
    - __unique_users__ - уникальные пользователи, работавшие с данной активностью
    - __unique_users_num__ - количество уникальных пользователей, работающих над данной активностью
    - __success_rate(...)__ - доля id, имеющих данную активность, которая выполнилась успешно (закончились успешными активностями)
    - __failure_rate(...)__ - доля id, имеющих данную активность, которая выполнилась неуспешно (закончились неуспешными активностями)
    
    
- IdMetric
    - __trace__ - цепочка (список активностей)
    - __trace_length__ - длина цепочки (кол-во активностей в цепочке)
    - __unique_activities__ - уникальные активности в цепочке
    - __unique_activities_num__ - количество уникальных активностей в цепочке
    - __loop_percent__ - процент зацикленности
    - __unique_users__ - уникальные пользователи, работающие с этим ID
    - __unique_users_num__ - кол-во уникальных пользователей, работавших с данным ID

- TraceMetric
    - __count__ - сколько раз данная цепочка встречается в логе
    - __ids__ - уникальные id с данной цепочкой
    - __trace_length__ - длина цепочки (кол-во активностей в цепочке)
    - __unique_activities__ - уникальные активности в цепочке
    - __unique_activities_num__ - количество уникальных активностей в цепочке активностей
    - __unique_users__ - уникальные пользователи, работающие над цепочкой активностей
    - __unique_users_num__ - количество уникальных пользователей, работающих над цепочкой активностей

 
- TransitionMetric
    - __count__ - сколько раз данный переход встречается в логе
    - __unique_ids__ - уникальные id  для каждого перехода
    - __unique_ids_num__ - количество уникальных id для каждого перехода
    - __aver_count_in_trace__ - среднее количество раз встречаемости объекта в цепочке
    - __loop_percent__ - процент зацикленности
    - __throughput__ - частота - количество выполненных переходов за единицу времени
    - __unique_users__ - уникальные пользователи, работающие над объектом
    - __unique_users_num__ - кол-во уникальных пользователей, работающих над объектом
    - __success_rate(...)__ - доля id, имеющих текущий переход, которые выполнились успешно (закончились успешными активностями)
    - __failure_rate(...)__ - доля id, имеющих текущий переход, которые выполнились неуспешно (закончились неуспешными активностями)
    
    
- UserMetric
    - __count__ - сколько раз данный пользователь встречается в логе
    - __unique_activities__ - уникальные активности, с которыми работал пользователь
    - __unique_activities_num__ - количество уникальных активностей, с которыми работал пользователь
    - __unique_ids__ - уникальные id с данным пользователем
    - __unique_ids_num__ - количество уникальных id с данным пользователем
    - __throughput__ - число раз выполнения объекта за единицу времени
    - __workload__ - доля активности лога, выполненных данным пользователем

### 1. ActivityMetric

In [ ]:
# Создание объекта ActivityMetric
activity_metric = ActivityMetric(data_holder, time_unit='d')

# Расчет всех метрик
activity_metric.apply().head()

### 2. TransitionMetric

In [ ]:
# Создание объекта TransitionMetric
transition_metric = TransitionMetric(data_holder, time_unit='d')

# Расчет всех метрик
transition_metric.apply().head()

### 3. IdMetric

In [ ]:
# Создание объекта IdMetric
id_metric = IdMetric(data_holder, time_unit='d')

# Расчет всех метрик
id_metric.apply().head()

### 4. TraceMetric

In [ ]:
# Создание объекта TraceMetric
trace_metric = TraceMetric(data_holder, time_unit='d')

# Расчет всех метрик
trace_metric.apply().head()

### 5. UserMetric

In [ ]:
# Создание объекта UserMetric
user_metric = UserMetric(data_holder, time_unit='d')

# Расчет всех метрик
user_metric.apply().head()

### Метрики + графы

В библиотеке реализована возможность представить ряд метрик на графе. Сделать это можно в классе `Graph` с помощью методов:
- __add_node_metric__ – добавить метрику, связанную с узлами графа
- __add_edge_metric__ – добавить метрику, связанную с ребрами графа

In [ ]:
# Расчет метрик
nodes_count_metric = activity_metric.count().to_dict()
edges_count_metric = transition_metric.count().to_dict()
mean_time_node_metric = activity_metric.mean_duration().fillna(0).to_dict()

# Получение графа из майнера
graph = causal_miner.graph

# Добавление метрик на граф
graph.add_node_metric('count', nodes_count_metric)
graph.add_edge_metric('count', edges_count_metric)
graph.add_node_metric('mean_time', mean_time_node_metric)

In [ ]:
# Создание объекта GraphvizPainter
painter = GraphvizPainter()

# Отрисовать граф и связать цвет узлов и ребер с нужными метриками
painter.apply(graph, node_style_metric='count', edge_style_metric='count')
# или painter.apply(graph, node_style_metric='mean_time', edge_style_metric='count')

# Сохранение графа
painter.write_graph("metric_graph.png", format = 'png')

# Отображение в jupyter-notebook
painter.show()

Чтобы удалить метрики с графа, следует воспользоваться следующими методами:
- __clear_node_metrics__ – удалить все метрики с нод (узлов)
- __clear_edge_metrics__ – удалить все метрики с ребер

In [ ]:
graph.clear_node_metrics()
graph.clear_edge_metrics()

## III. Conformance Checking

### TokenReplay

`TokenReplay` позволяет рассчитать *fitness*, который показывает, насколько хорошо граф описывает бизнесс-процесс (1 – хорошо, 0 – плохо). Fitness вычисляется отдельно для каждой цепочки (id) при ее проигрывании по сети Петри по следующей формуле:
$$ Fitness = \frac{1}{2}\Big(1-\frac{missed}{consumed}\Big) + \frac{1}{2}\Big(1-\frac{remaining}{produced}\Big) $$
- produced tokens – появились в результате перехода
- consumed tokens – удалились в результате перехода
- remaining tokens – остались в конце проигрывания
- missing tokens – не было, но они необходимы для проигрывания, поэтому их вставляют

Библиотека выдает следующие метрики:
- значения 4 величин и fitness каждой цепочки
- усредненный fitness по всем цепочкам (__mean_fitness__)
- fitness по всему логу – в формулу подставляются суммарные значения 4 величин по всем цепочкам в логе (__average_fitness__)

In [ ]:
from sberpm.conformance_checking import TokenReplay

In [ ]:
token_replay = TokenReplay(data_holder, alpha_miner.graph)
token_replay.apply()
token_replay.result

In [ ]:
print('mean:', token_replay.mean_fitness)
print('average:', token_replay.average_fitness)

Также в библиотеке доступен более общий класс ConformanceChecking, сдержащий TokenReplay и рад других метрик:
- precision
- generalization
- simplicity

In [ ]:
from sberpm.conformance_checking import ConformanceChecking

In [ ]:
cc = ConformanceChecking(data_holder, alpha_miner.graph)
cc.get_conformance_checking()

In [ ]:
cc.get_fitness_df()

## IV. BPMN

Для сохранения графа в формате BPMN (Business Process Model and Notation) можно воспользоваться `BpmnExporter` из модуля __`sberpm.bpmn`__. Он имеет следующие методы:
- __apply_petri__ – построить BPMN для сети Петри
- __get_string_representation__ – получить BPMN-нотацию графа
- __write__ – записать граф в BPMN формате

На данный момент сохранять можно только графы, полученные из Alpha Miner.

In [ ]:
from sberpm.bpmn import BpmnExporter

In [ ]:
bpmn_exporter = BpmnExporter()
bpmn_exporter.apply(alpha_miner.graph)
bpmn_exporter.get_string_representation()[:1000]

In [ ]:
bpmn_exporter.write('exported.bpmn')

Для загрузки BPMN-файла есть класс `BpmnImporter` со следующими методами:
- __load_bpmn_from_xml__ – загрузить граф, представленный в виде BPMN
- __get_pydotplus_graph__ – получить граф в формате pydotplus

In [ ]:
from sberpm.bpmn import BpmnImporter

In [ ]:
bpmn_importer = BpmnImporter()
bpmn_importer.load_bpmn_from_xml('exported.bpmn')
pydot_graph = bpmn_importer.get_pydotplus_graph()
pydot_graph.write('imported_bpmn.svg', prog='dot', format='svg')

## V. Визуализация

Класс `ChartPainter` из модуля __`sberpm.visual`__ предназначен для создания основных типов графиков. В основе визуализации лежит библиотека __`plotly`__, благодаря чему все диаграммы являются интерактивными. 

In [ ]:
from sberpm.visual import ChartPainter

Параметры:
- __data__ – данные, которые необходимо визуализировать (DataFrame, DataHolder или объекта класса метрик)
- __template__ – стиль графиков, по умолчанию _plotly_
- __palette__ – цветовая палитра графиков, по умолчанию _sequential.Sunset_r_

Каждый метод `ChartPainter` позволяет отрисовать график определенного типа:
- __hist__ – гистограмма
- __bar__ – столбчатая диаграмма
- __box__ – ящичковая диаграмма
- __scatter__ – диаграмма рассеяния
- __line__ – линейный график
- __pie__ – круговая диаграмма
- __sunburst__ – диаграмма солнечные лучи
- __heatmap__ – 2D гистограмма
- __timeline__ – диаграмма Ганта
- __pareto__ – диаграмма Парето

Основные параметры методов (для более подробной информации см. документацию):
- __x__, __y__ – названия столбцов для отрисовки по осям X и Y соответственно
- __sort__ – название столбца для сортировки значений
- __n__ – количество строк для визуализации
- __color__ – название столбца для задания цвета элементам графика
- __subplots__ – кортеж вида (rows, cols, ncols), где rows и cols – это названия столбцов для отрисовки нескольких графиков по рядам и столбцам соответственно, а ncols – это количество столбцов
- __text__ – название столбца с текстовой информацией (или ее вид) для отображения на графике
- __orientation__ – ориентация графика
- __opacity__ – прозрачность элементов графика
- __edge__ – границы элементов графика
- __title__ – название графика

Каждый метод прост в использовании, но при этом обладает достаточно широким функционалом, который позволяет построить графики для любых задач.

### Гистограмма

In [ ]:
painter = ChartPainter(id_metric)
painter.hist(x='total_duration', edge=True)

### Столбчатая диаграмма

In [ ]:
painter = ChartPainter(user_metric)
painter.bar(x=data_holder.user_column, y='total_duration', text=True)

### Диаграмма рассеяния

In [ ]:
painter = ChartPainter(id_metric)
painter.scatter(x='mean_duration', y='median_duration', color='unique_users_num', size='trace_length', 
                edge=True, opacity=0.8)

### Круговая диаграмма

In [ ]:
painter = ChartPainter(user_metric)
painter.pie(labels='count', n=15)

### Гистограмма распрелделения активностей по диапазонам времени

##### По всем активностям

In [ ]:
painter = ChartPainter(data_holder)
painter.hist_activity_of_dur(top= False, use_median=False)

##### По топ активностям

In [ ]:
painter.hist_activity_of_dur(top= True)

##### По одной активности

In [ ]:
painter.hist_activity_of_dur(by_activity='Stage_6')

# ----------Машинное обучение----------

## Проверка наличия моделей

import sberpm.models
Данный модуль используется для проверки __необходимых__ моделей библиотеки, без которых __не будут работать__ некоторые модули представленные ниже.

Модели можно скачать с репозитория на __BitBucket__.

В данный момент в библиотеке используются следующие модели:
- __Navec__
- __PM__
- __Bert__


В модуле хранятся следующая функция:
- __check_contained_models()__ - с помощью этой функции можно проверить имеющиеся модели.
- __get_models_path()__ - c помощью этой функции можно узнать путь к файлу библиотеки, и конкретно к папке, в которой должны храниться соотвествующие модели.

In [ ]:
import sberpm.models as check

In [ ]:
check.check_contained_models()

In [ ]:
check.get_models_path()

## I. Кластеризация этапов

Данный модуль служит для кластеризации этапов процесса, для нахождения близких или идентичных этапов процессов. 

Рассматриваются только уникальные названия этапов процессов. Для кластеризации используется лемматизация алгоритмами библиотеки __Natasha__ всех слов с последующим отображением этапа процесса с помощью нейронной сети Navec в 300-мерное пространство, на основании которого можно рассчитать попарные расстояния Word Mover's Distance между этапами процессов, полученные данные образуют матрицу расстояний. На основании этой матрицы производится кластеризация методом __DBSCAN__. Полученные кластеры образуют близкие этапы процесса.

После кластеризации выбираются кластеры с неотрицательными номерами. Каждый такой кластер указывает на схожие этапы процесса. -1 кластер указывает на этапы, на которые нет похожих.

Параметры передаваемые в класс __StagesClustering__:
- __data (SberPM DataHolder или pandas DataFrame)__ - класс с хранящимися данными.
- __stages_col (str)__ - столбец данных, содержащий текстовые названия этапов процесса.
- __generalizing_ability (float = default 0.5)__ - Коэффициент обобщающей способности (указывается в пределах [0;1]). Чем ниже будет коэффициент, тем больше будет кластеров.
- __type_model_w2v (bool = default True)__ - выбор модели:
    - True- общая модель
    - Fasle - модель специализированная для сбербанка

Методы класса __StagesClustering__:
- __apply()__ - запустит процесс анализа.
- __get_clustered_result()__ - вернёт pandas DataFrame с указанием кластера каждого этапа.
- __get_result_data()__ - вернёт исходный pandas DataFrame с добавленной колонкой 'clusters'.

In [ ]:
from sberpm.ml.stages_clustering import StagesClustering

In [ ]:
Stage_clust = StagesClustering(data=data_holder, stages_col='some_text')
Stage_clust.apply()

In [ ]:
Stage_clust.get_clustered_result()

In [ ]:
Stage_clust.get_result_data()

##  II. Автоматический поиск неэффективностей

Модуль автоматического поиска неэффективностей __`sberpm.autoinsights`__ позволяет в автоматическом режиме выявить __слабые места и уязвимости процесса__ и наглядно продемонстрировать их на графе процесса. При анализе классом `AutoInsights` учитываются такие факторы, как:
1. Длительность этапа
2. Рост длительности этапа
3. Нерегулярность (редкость) этапа
4. Зацикленность
5. Этап имеет bottleneck c низкой вариативностью
6. Этап имеет bottleneck c высокой вариативностью
7. Этап имеет большую длительность из-за частых повторений инцидентов
8. Этап имеет большую длительность из-за разовых инцидентов
9. Этап приводит к росту времени процесса и/или прочих этапов
10. Этап проходит с ошибками, что приводит к замедлению процесса
11. Этап проходит с критическими ошибками системы, что приводит к неуспеху процесса
12. Этап проходит со структурными ошибками, что приводят к неуспеху процесса
13. Сторнирование на данном этапе приводит к неуспеху замедлению процесса
14. Сторнирование на данном этапе приводит к неуспеху процесса
15. Уровень аномальности
16. Сумма финансовых эффектов

Общим итогом вычислений являются следующие два параметра:
- __Уровень аномальности - *[0, 1]*__

    Для каждого объекта (активность и ребро) считается его уровень аномальности - метрика со значениями от 0 до 1 включительно. Чем больше уровень аномальности, тем больше инсайтов может дать объект.
- __Сумма финансовых эффектов - *[0, +inf)*__
    
    Сумма финансовых эффектов представляет собой суммы финэффектов по каждой метрике в таблице, получаемой через *get_clustered_result*. Финэффект рассчитывается для каждой активности на основе стоимости 'секунды работы человека' *sec_cost*. Также, в зависимости от метрики, на финэффект влияют продолжительности этапов, циклы и другие проблемы активности или активностей, от которой зависит текушая активность. 

AutoInsights имеет следующие параметры:
- **data_holder (SberPM DataHolder)** – экземпляр класса DataHolder с хранящимися данными
- **success_activity (string = default None)** - этап представляющий успешный процесс
- **clust_eps (float = default 0.1)** - обообщенный гиперпараметр для DBSCAN и IsolationForest
- **sec_cost (float = default 0.01)** - стоимость минуты человеческой работы

AutoInsights имеет следующие методы:
- **apply()** – выполняет расчет неэффективностей во входном логе.
- **get_result()** – возвращает таблицу инсайтов виде отнормированных значений для каждого фактора.
- **get_clustered_result()** – возвращает таблицу кластеризованных метрик инсайтов для активностей.
- **get_boolean_clustered_result()** – возвращает таблицу кластеризованных метрик инсайтов для активностей, со значениями True/False.
- **get_description()** - возвращает текстовое описание ненулевых финэффектов по таблице кластеризованных метрик инсайтов.
- **get_transition_ai()** – возвращает таблицу уровней аномальности для переходов между парами активностей.

In [ ]:
from sberpm.autoinsights import AutoInsights

auto_i = AutoInsights(data_holder)
auto_i.apply()

Таблица метрик инсайтов

In [ ]:
auto_i.get_result()

Таблица кластеризованных метрик инсайтов

In [ ]:
auto_i.get_clustered_result()

Таблица кластеризованных инсайтов с метриками True/False

In [ ]:
auto_i.get_boolean_clustered_result()

Текстовое описание ненулевых финэффектов по таблице кластеризованных метрик инсайтов

In [ ]:
print(auto_i.get_description())

Таблица уровней аномальности для переходов с этапа на этап 

In [ ]:
auto_i.get_transition_ai()

### Визуализация инсайтов на графе

Для визуализации следует воспользоваться методом **apply_insights** классов `sberpm.visual.GraphvizPainter` или `sberpm.visual.MlPainter`. Нужно подать :
- **граф** для отрисовки (на нём может быть меньше нод/рёбер, чем в таблицах автоинсайтов), 
- **объект `AutoInsights`**, у которого был вызван метод apply
- параметры отрисовки

In [ ]:
from sberpm.miners import heu_miner

graph = heu_miner(data_holder, 0.6)

In [ ]:
painter = GraphvizPainter()
painter.apply_insights(graph, auto_i)
painter.show(True)

На графе нет чисто красных рёбер, так как HeuMiner вернул граф, на котором присутствуют не все рёбра, то есть "инсайтных" рёбер на графе просто нет. Можно получить граф со всеми возможными рёбрами при помощи SimpleMiner, но он будет слишком большой.

## III. Поиск аномалий

__Выявление аномалий__ (также __обнаружение выбросов__) — это распознавание во время интеллектуального анализа данных редких данных, событий или наблюдений, которые вызывают подозрения ввиду существенного отличия от большей части данных.

Для поиска аномалий (выбросов) в данных в библиотеке есть модуль __`sberpm.ml.anomaly_detection`__, в котором есть классы `OutlierCBLOF`, `OutlierForest`, `OutlierLOF`, `OutlierOCSVM`, `OutlierCustom`, `OutlierEnsemble`. В каждом классе реализован свой алгоритм __выявления аномалий без учителя__, который обнаруживает аномалии в непомеченных наборах данных при предположении, что большая часть набора данных нормальна, путем поиска представителей, которые меньше подходят к остальному набору данных. 

`OutlierEnsemble` это композиция алгоритмов обнаружения аномалий, итоговой ответ которой представляет собой голосование (объект считается выбросом, если большинство алгоритмов определило его как выброс) следующих алгоритмов: [KNN](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.knn), [ABOD](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.abod), [HBOS](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.hbos), [Isolation Forest](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.iforest).

In [ ]:
from sberpm.ml.anomaly_detection import OutlierCBLOF, OutlierForest, OutlierLOF, \
                                        OutlierOCSVM, OutlierCustom, OutlierEnsemble

В качестве параметра объект принимает на вход DataHolder, по которому рассчитывает базовые статистики, такие как среднее время, длина цепочки активностей, число уникальных пользователей (если они есть) и т. д. 

In [ ]:
outlier_detector = OutlierForest(data_holder)

Каждый класс имеет следующие методы:
- __add_feature__ – добавление признака, по которому требуется найти аномалии 
- __add_groupby_feature__ – добавление признака для поиска аномалий, рассчитанного по сгруппированным данным 
- __apply__ – запуск алгоритма
- __get_outlier_ids__ – вывод id аномальных процессов
- __print_result__ – вывод статистики по аномалиям
- __show_permutation_importance__ – иллюстрация permutation importance признаков, по которым отличаются выбросы (работает везде кроме `OutlierEnsemble`)

In [ ]:
# Добавление признака с именем max_time, вычисляемого путем применения функции max к колонке 
# data_holder.duration_column в сгруппированных по id данных (масимальное время активности в процессе)
outlier_detector.add_groupby_feature('max_time', data_holder.duration_column, max)

В модуле реализовано 5 техник выявления аномалий:
1. __Isolation Forest (IF)__
2. __One-Class Support Vector Machines (OCSVM)__
3. __Local Outlier Factor (LOF)__
4. __Cluster-Based Local Outlier Factor (CBLOF)__
5. `OutlierEnsemble` в котором есть __KNN__, __HBOS__, __ABOD__, __Isolation Forest__

Также можно использовать любой другой алгоритм поиска аномалий (например, из библиотеки __pyod__).

### Isolation Forest

> В основе техники __изолирующего леса__ лежит идея о том, что аномальные наблюдения легче отделить от остальных (нормальных) объектов датасета. Алгоритм строит ансамбль изолирующих бинарных деревьев решений, в каждом узле которого выбор признака и порога разбиения производится случайным образом. Дерево строится до тех пор, пока в листе не останется только один объект или объекты с одинаковыми значениями. Интуитивно понятно, что __аномальные__ точки – это те, что имеют меньшую длину пути в дереве, которая определяется как число ребер, которые объект проходит от корневого узла до листа. 

In [ ]:
outlier_detector = OutlierForest(data_holder)

### One-Class Support Vector Machines

> Основная идея классического __метода опорных векторов (SVM)__ заключается в разделении объектов, относящихся к разным классам, гиперплоскостью так, чтобы максимизировать расстояние между ними. Алгоритм __OCSVM__, как следует из названия, обучается на данных, принадлежащих одному классу – классу нормальных объектов. Он определяет границы этих объектов и классифицирует все остальные точки, лежащие по другую сторону от разделяющей поверхности, как __аномальные__.

In [ ]:
outlier_detector = OutlierOCSVM(data_holder)

### Local Outlier Factor

> __Локальный уровень выброса (LOF)__ основывается на концепции локальной плотности объекта, где локальность задается его $k$ ближайшими соседями, расстояния до которых используются в качестве оценки плотности. Путем сравнения локальной плотности объекта с локальной плотностью его соседей, можно выделить области с аналогичной плотностью и точки, которые имеют существенно меньшую плотность, чем ее соседи. Эти точки считаются __выбросами__.

In [ ]:
outlier_detector = OutlierLOF(data_holder)

### Cluster-Based Local Outlier Factor

> В отличие от стандартного LOF, основанном на метрическом подходе к выявлению локальных выбросов, __CBLOF__ выявляет кластерную структуру данных, разделяет кластеры на "большие" и "малые" и затем определяет локальность малых кластеров по отношению к большим. Кластеры, чья локальность по отношению к другим мала, определяются как __выбросы__.

In [ ]:
outlier_detector = OutlierCBLOF(data_holder)

Помимо указанных 4 алгоритмов поиска аномалий, можно воспользоваться любым другим, который не встроен в библиотеку, но есть в pyod – например, __histogram-based outlier detection (HBOS)__.

In [ ]:
from pyod.models.hbos import HBOS

hbos = HBOS(contamination=0.1)
outlier_detector = OutlierCustom(data_holder, hbos, outlier_label=1)

После выбора алгоритма, его следует применить с помощью метода __apply__.

In [ ]:
outlier_detector.apply()

Результаты представляют собой список id аномалий (метод __get_outlier_ids__), таблицу с описательными статистиками по аномальным и нормальным объектам (метод __print_result__), а также графическую иллюстрацию важности использованных для поиска аномалий признаков (метод __show_permutation_importance__). 

In [ ]:
outlier_detector.get_outlier_ids()

In [ ]:
outlier_detector.print_result()

In [ ]:
outlier_detector.show_permutation_importance()

### `OutlierEnsemble`
> `OutlierEnsemble` это композиция алгоритмов обнаружения аномалий, итоговой ответ которой представляет собой голосование (объект считается выбросом, если большинство алгоритмов определило его как выброс) следующих алгоритмов: [KNN](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.knn), [ABOD](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.abod), [HBOS](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.hbos), [Isolation Forest](https://pyod.readthedocs.io/en/latest/pyod.models.html#module-pyod.models.iforest). В качестве алгоритмов можно выбрать любое подмножетво из {"HBOS", "ABOD", "KNN", "IForest"}.

In [ ]:
outlier_detector = OutlierEnsemble(data_holder, ["HBOS", "ABOD", "KNN", "IForest"])

In [ ]:
outlier_detector.apply()

In [ ]:
outlier_detector.print_result()

## IV. Факторный анализ

__Факторный анализ__ – это многомерный метод, применяемый для изучения взаимосвязей между значениями переменных. Предполагается, что известные переменные зависят от меньшего количества неизвестных переменных и случайной ошибки. С помощью факторного анализа возможно выявление скрытых переменных факторов, отвечающих за наличие линейных статистических корреляций между наблюдаемыми переменными.

Параметры передаваемые в класс __FactorAnalysis__:
- __data_holder (SberPM DataHolder)__ - класс с хранящимися данными
- __target_column (str)__ - целевая колонка относительно которой рассчитываются значения $R^2$
- __type_of_target (str)__ - Определяет тип целевой колонки. Возможные значения: **number** - числовой, **string** - строковый, **time** - временной.
- __categorical_cols (List[str])__ - список столбцов с **категориальными** значениями
- __numeric_cols (List[str])__ - список столбцов с **числовыми** значениями
- __date_cols (List[str])__ - список столбцов с форматом данных - **время**
- __extended_search (Bool)__ - флаг расширенного анализа, т.е. будут применяться дополнительные методы анализа данных. Значение по умолчанию - **False**
- __count_others (Bool)__ - расчёт колонки **Прочее**. Значение по умолчанию - **False**

Для корректной работы класса необходимо передать название target_column и хотябы один из параметров categorical_cols, numeric_cols или date_cols по которым и будут считаться значения $R^2$.

Методы класса **FactorAnalysis**:
- __apply()__ - запустит процесс анализа и вернёт результат в виде таблицы (pandas DataFrame)
- __get_result()__ - вернёт результат работы факторного анализа, в случае если ранее был запущен метод **apply()**

In [ ]:
from sberpm.ml.factor_analysis import FactorAnalysis

In [ ]:
dh = DataHolder('InternationalDeclarations.csv', 'id', 'concept:name', 'time:timestamp')
res = FactorAnalysis(data_holder=dh,
                     target_column='case:AdjustedAmount',
                     type_of_target="number",
                     categorical_cols=['org:resource', 'case:Permit ActivityNumber','org:role'],
                     numeric_cols=['case:Amount','case:RequestedAmount','case:Permit RequestedBudget'],
                     date_cols=['time:timestamp'],
                     extended_search=True,
                     count_others=True,
                     )

In [ ]:
res.apply()

## V. Рекомендательная система

Данный модуль представляет собой рекомендательную систему, ранжирующую этапы процесса в порядке их приоритетности для реинжениринга. Система показывает, на какие активности нужно обратить внимание в первую очередь при оптимизации процесса. Сначала выбирается лучшая модель, описывающую зависимость таргетной метрики (metric) каждой из Активностей от признаковых метрик (metric_f) всех остальных Активносетй. В качестве метрик могут выступать количество рециклов (количество появлений активности, __"appearance"__), время выполнения(__"time"__), количество повторений (__"recycles"__), кастомная метрика пользователя ("__user_metric__"). Далее на основании метрики строятся коэффициенты проблемности, в порядке убывания которых ранжируются Активности.

Параметры передаваемые в класс __RecommendSystem__:
- __data (SberPM DataHolder)__ - класс с хранящимися данными.
- __mode (str = default 'best')__ - настройка качества анализа. Может принимать 2 параметра:
    - __'best'__ - провести точный, но долгий расчёт
    - __'fast'__ - менее точный, но быстрый рассёт
- __metric (str = default 'appearance')__ - целевая метрика ранжирования. Может принимать следующие значения: 
    - __'appearance'__ - количество появлений Активности;  
    - __'time'__ - общее время активности, 
    - __'recycles'__ - количество рециклов активности;
    - __'user_metric'__ - кастомная метрика пользователя.
- __metric_f (str = default 'appearance')__ - признаковая метрика, единица измерения независимых Активностей. Может принимать следующие значения: 
    - __'appearance'__ - количество появлений Активности;  
    - __'time'__ - общее время активности, 
    - __'recycles'__ - количество рециклов активности;
    - __'user_metric'__ - кастомная метрика пользователя.
    
- __user_metric_column__ (str = default None) - название колонки данных, где находится кастомная метрика пользователя.
- __user_meric_column_f__ (str = default None) - название колонки данных, где находится признаковая кастомная метрика пользователя.
- __auto_i__ (bool = default False) - с учётом уровня проблемности.

Методы класса __RecommendSystem__:
- __apply()__ - запустит процесс анализа.
- __get_result()__ - вернёт pandas DataFrame с указанием кластера каждого текстового поля.

In [ ]:
from sberpm.ml.recommendation_system import RecommendSystem

In [ ]:
Rec_syst = RecommendSystem(data=data_holder, metric='time')
Rec_syst.apply()

In [ ]:
Rec_syst.get_result()

###### Рекомендательная система + автоинсайты

In [ ]:
Rec_syst = RecommendSystem(data=data_holder, metric='time', auto_i=True)
Rec_syst.apply()
Rec_syst.get_result()

## VI. Анализ текстов

Данный модуль служит для кластеризации текстов. 
Для каждого кластера алгоритм выдает __номер кластера__ (название кластера или самое близкое сообщение к центроиду кластера), и 10 самых распространенных слов в кластере. Для кластеризации используется отображение текста с помощью нейронной сети word2vec в 300-мерное пространство, дальше используется метод понижения размерности с минимальной потерей информации с помощью алгоритма PCA, полученные данные кластеризуются методом DBSCAN.

Параметры передаваемые в класс __TextClustering__:
- __data (SberPM DataHolder или pandas DataFrame)__ - класс с хранящимися данными.
- __description (str)__ - название колонки, в которой находятся текстовые данные.
- __pca_dim (str = default 'medium')__ - режим работы. Может принимать значения 'fast', 'medium', или 'full_quality'.
- __type_model_w2v (bool = default True)__ - выбор модели:
    - True- общая модель
    - Fasle - модель специализированная для сбербанка
- __dbscan_eps (float = default 0.5)__ - максимальное расстояние между двумя образцами, чтобы один из них рассматривался как находящийся по соседству с другим.
- __min_samples (int = default 2)__ - минимальное количество текстов в кластере, минимальное 2.
- __only_unique_descriptions (bool = default False)__ - флаг 'True' оставляет только уникальные записи.
- __cluster_marking (List[str] = default None)__ - возможность задать названия нескольких кластеров, к которым будут отнесены максимально близкие записи, а оставшиеся нерелевантные записи также будут кластеризованы.

Методы класса __TextClustering__:
- __apply()__ - запустит процесс анализа.
- __get_result()__ - вернёт pandas DataFrame с указанием кластера каждого текстового поля.

In [ ]:
from sberpm.ml.text_clustering import TextClustering

In [ ]:
Text_Clust = TextClustering(data=data_holder, description='some_text', pca_dim='full_quality')
Text_Clust.apply()

In [ ]:
Text_Clust.get_result()

## VII. Сентиментный анализ

Данный модуль представляет собой систему для анализа тональности словесных комментариев в текстовом поле. Модуль анализа тональности имеет два режима: «базовый» и «продвинутый». В «базовом» режиме тональность текста определяется как «positive» или «negative», численное значение тональности определяется в пределах от -1 до 1 (от негатива к позитиву). Границу двух смежных интервалов можно настроить.

В «продвинутом» режиме проводится более точная оценка тональности текста, которая определяется как «positive», «neutral» или «negative»,  численное значение тональности определяется в пределах от -1 до 1 (от негатива к позитиву). Границы трех смежных интервалов  в пределах от -1 до 1, соответствующих пользовательским уровням можно настраивать.

Параметры передаваемые в класс __SentimentAnalysis__:
- __data (SberPM DataHolder или pandas DataFrame)__ - класс с хранящимися данными.
- __text_column (str)__ - название колонки, в которой находятся текстовые данные.
- __mode (bool = default False)__ - выбор режима:
    - True - «продвинутый», точный режим (для работы необходима модель bert_model)
    - Fasle - «базовый» режим работы
- __lower_bound (float = default None)__ - нижняя смежная граница двух интервалов. В случае если используется «продвинутый» режим, то задаётся нижняя смежная граница для трёх интервалов.
- __upper_bound (float = default None)__ - верхняя смежная граница двух интервалов. Используется в случае использования «продвинутого» режима.


Методы класса __SentimentAnalysis__:
- __apply()__ - запустит процесс анализа и вернет результат pandas DaraFrame.
- __get_result()__ - вернёт результат работы в виде pandas DataFrame.



In [ ]:
from sberpm.ml.sentimental_analysis import SentimentAnalysis

###### Базовый режим

In [ ]:
df  = pd.DataFrame(data = ['Все плохое 😥', 'Терпимо' , 'Все хорошее', 'Гуд', 'Ужасно🤦‍♀️', 'nice', 'I want pizza',
                           '✔', 'Нормально', 'Плохо!! (╯°□°）╯︵ ┻━┻', 'disgusting service'],
                   columns = ['text'])
Sent_analysis = SentimentAnalysis(data = df, text_column='text')

In [ ]:
Sent_analysis.apply()

###### Продвинутый режим

In [ ]:
Sent_analysis = SentimentAnalysis(data = df, text_column='text', mode=True)

In [ ]:
Sent_analysis.apply()

## VIII. Поиск счастливого пути

Задачу поиска счастливого пути можно решить с помощью обучения с подкреплением (RL), которое по своей сути адаптировано к поиску оптимальных действий. RL работает с двумя объектами: агентом и средой. Во время обучения агент взаимодействует с окружающей средой, совершает действия и получает обратную связь, которая называется вознаграждением. Задача формулируется в рамках марковского процесса принятия решений, который основан на:
* множестве состояний в мире 
* множестве действий 
* вероятностном распределении следующего состояния при условии текущего состояния и завершенного действия 
* награды при переходе между состояниями при выполнении действий. 

Выполнение марковского свойства состоит в том, что следующее состояние условно не зависит от прошлых состояний и действий, учитывая текущее состояние и действие. Граф процесса рассматривается как среда, состояния — узлы графа (активности), действия — ребра (выбор следующей активности для перехода), награда — среднее отрицательное время перехода между прошлым и настоящим состояниями. При наличии ключевых состояний за переход в них также начисляется награда. Цель состоит в том, чтобы выбрать оптимальную политику — отображение из пространства состояний в пространство действий или, другими словами, руководство к действию в каждом состоянии — которое максимизирует ожидаемую дисконтированную сумму вознаграждений, проходящих через граф процесса.
Оптимальная политика и, как следствие, путь находятся при помощи AutoRL с использованием лучшего по дисконтированной сумме наград  метода из:  value iteration,  Q-learning, cross entropy, genetic algorithm.

Параметры передаваемые в класс __HappyPath__:
- __data_holder (SberPM DataHolder)__ - класс с хранящимися данными
- __key_node (str)__ - должен быть один элемент – ключевой узел
- __initial_state (str = default "startevent")__ - точка старта пути 
- __reward_for_key (float = default 10.0)__ - награда за ключевой узел
- __reward_for_end (float = default 0.0)__ - награда за окончание пути
- __prob_increase (float = default 0.5)__ - увеличение вероятности попасть в узел при выборе этого узла с последующей нормировкой
- __clear_outliers (float = default 0.05)__ - переходы с вероятностью меньшей, чем эта, не учитываются. Значение должно быть в диапазоне [0,1]
- __short_path (int = default None)__ - после какого достижения ключевого узла на пути прекращать давать награду за ключевой узел
- __gamma (float = default 1)__ - дисконтирующий фактор
- __mode (str = default "complete")__ - количество RL алгоритмов для решения задачи. Принимает значения: short, normal, long, complete.
- __regime (str = default "static")__ - меняется ли среда или нет после очередного достижения ключевого узла. Принимает значения 'dynamic' или 'static'.
- __penalty (float = default 0.0)__ - штраф за каждый шаг
- __output_algo_params__ (bool = default false) - добавить в вывод параметры выбранного алгоритма 
- __mut_list_gen (List[float] = default [0.1])__ - вероятность мутации в генетическом алгоритме
- __children_list_gen (List[int] = default [100])__ - численность популяции в генетическом алгоритме
- __iters_list_gen (List[int] = default [400])__ - количество популяций в генетическом алгоритме
- __percentile_list_cem (List[int] = default [20])__ - квантиль элитных сессий в алгоритме кросс энтропии
- __learning_list_cem (List[float] = default [0.1])__ - скорость обучения в алгоритме кросс энтропии
- __iters_list_cem (List[int] = default [800])__ - количество итераций в алгоритме кросс энтропии
- __alpha_list (List[float] = default [0.15])__ - скорость обучения в алгоритме Q-learning
- __epsilon_list_q (List[float] = default [0.1])__ - вероятность epsilon-greedy policy в алгоритме Q-learning
- __iters_list_q (List[int] = default [4000])__ -количество итераций в алгоритме Q-learning
- __iters_list_vi (List[int] = default [50])__ - количество итераций в алгоритме value iteration
- __num_iter_list_vi (List[int] = default [250])__ - максимум повторений до сходимости в алгоритме value iteration


Методы класса __HappyPath__:
- __apply()__ - запустит процесс анализа
- __get_df_rl()__ - вернёт результат с указанием лучшего пути, количеством награды (rewards), и успешность прохождения в виде таблицы DataFrame.
- __get_best_algo()__ - вернёт имя лучшего алгоритма
- __get_best_path()__ - вернёт счастливый путь
- __get_best_rew()__ - вернёт количество наград лучшего алгоритма


In [ ]:
from sberpm.ml.happy_path import HappyPath

In [ ]:
dfna = pd.read_csv('log_happy_path.csv',
                   encoding = 'windows-1251', sep = ';')
dh_for_HP = DataHolder(dfna, id_column = 'ID_PRODUCT', activity_column = 'EVENT_NAME', 
                          start_timestamp_column = 'EVENT_DATE')

In [ ]:
HP = HappyPath(dh_for_HP, key_node = 'Печать договора', output_algo_params=True)

In [ ]:
HP.apply()

In [ ]:
HP.get_df_rl()

In [ ]:
HP.get_best_algo()

In [ ]:
HP.get_best_rew()

In [ ]:
HP.get_best_path()

### Визуализация HappyPath на графе

Для визуализации следует воспользоваться методом **apply_happy_path** классов `sberpm.visual.GraphvizPainter`. 
Нужно подать :
- **граф** для отрисовки (на нём может быть меньше нод/рёбер, чем в таблицах автоинсайтов), 
- **объект `HappyPath`**, у которого был вызван метод apply
- __hide_disconnected_nodes (bool = default False)__ - флаг отрисовки изолированных узлов (узлов без входных и выходных переходов). Если False, то такие узлы отображаться не будут.
- __edges_widths (float = default 3.0)__ - толщина стрелок перехода HappyPath.

In [ ]:
simpleminer = SimpleMiner(dh_for_HP)
simpleminer.apply()
graph = simpleminer.graph

# Создание объекта GraphvizPainter
painter = GraphvizPainter()

# Применение HappyPath
painter.apply_happy_path(graph, HP)

# Можно вывыести граф в notebook
painter.show()

## IX. Предсказание структуры графа

Класс __GSPredictor__ (graph structure predictor) содержит алгоритм предсказания структуры графа, а именно, предсказываются две величины: вероятности и средние времена выполнения нод и рёбер графа. Эти величины представляются как временные ряды, получаемые из имеющихся данных, далее используются ml- и специализированные для работы с временными рядами алгоритмы для предсказания.

Шаги алгоритма:

    I. Подготовка данных

    1. Расчёт временных рядов для нод и рёбер
        a) вероятности нод/рёбер
           (сумма вероятностей нод/рёбер в одном временном промежутке равна 1;
            или 0 если в данном временном промежутке не было ни одной ноды/ни одного ребра)
        b) средние времена выполнения нод/рёбер

    2. Превращение врем. рядов в матрицы лагов и добавление дополнительной информации:
        - разница лаг-колонок
        - one-hot encoding месяцев (если одна точка временного ряда представляет собой месяц или год)
        - число рабочих дней в месяце (если одна точка временного ряда представляет собой месяц)

    II. Предсказание (следующие шаги выполняются для каждого врем. ряда/матрицы лагов)

    0. Данные разделяются на 3 части: "train", "test", "val".
    1. Используется перебор параметров для поиска лучшей ml-модели на 
       train/test данных.
    2. Используются несколько алгоритмов отборов признаков для поиска лучших признаков на train/test данных. Здесь используется        ml-модель, найденная на предыдущем шаге.
    3. Выполняется шаг 1 (поиск лучшей модели) с некоторыми отличиями:
        a) используется больше моделей: те же ml-модели + Sarimax и Holt
        b) другие данные используются для обучения/тестирования: train+test/val
        c) используются только "лучшие признаки" данных, найденные на предыдущем шаге
    4. Признаки из шага 2 (и 3) и модель из шага 3 используются для предсказания результата.

__Параметры__
​
- __data_holder__: DataHolder
​
- __test_size: float, default=0.1__<br>&nbsp;
    Размер "тест" части, даля от общего объёма данных.
​
- __val_size: float, default=0.1__<br>&nbsp;
    Размер "val" части, даля от общего объёма данных.
​
- __pred_period: int, default=5__<br>&nbsp;
    Кол-во точек временного ряда, которые нужно предсказать.
​
- __period_type: {'D', 'M', 'Y'}__<br>&nbsp;
    Временной промежуток одной точки временного ряда (гранулярность): день, месяц, год.
​
- __two_models: bool, default=False__<br>&nbsp;
    Если True, также перебрать все возможные комбинации из двух ml-моделей и объеденить их результат.
​
- __edges: bool, default=False__<br>&nbsp;
    Если False, делать предсказания только для нод.
    Если True, для нод и рёбер.
​
- __refit: bool, default=False__<br>&nbsp;
    Если True, переобучать модель на каждом шаге при финальном предсказании,
    используя имеющиеся данние и те, которые уже предсказаны.
​
- __quick_mod: bool, default=False__ <br>&nbsp;
    быстрый режим прогноза.

__Методы__ (для отображения результата)

nodes/edges - ноды/рёбра<br>
prob/duration - вероятность/среднее время выполнения

- plot_nodes_prob()
- plot_nodes_duration()
- plot_edges_prob()
- plot_edges_duration()



- __get_predicted_graph()__ - отрисует спрогнозированный SimpleMiner граф из последнего прогноза только с узлами, которые имеют вероятность больше, чем base_probability.
    - __base_probability__ - пороговое значение вероятности узла. Если если значение вероятности будет меньше заданного, то узел не будет отрисован.

__Аттрибуты__

после работы алгорима временные ряды содержатся внутри класса

pred - предсказанные временные ряды<br>
без pred - врем. ряды, построенные на имеющихся данных

- nodes_prob: pd.DataFrame
- nodes_duration: pd.DataFrame
- edges_prob: pd.DataFrame
- edges_duration: pd.DataFrame

- nodes_prob_pred: pd.DataFrame
- nodes_duration_pred: pd.DataFrame
- edges_prob_pred: pd.DataFrame
- edges_duration_pred: pd.DataFrame

In [ ]:
from sberpm.ml.graph_structure_prediction import GSPredictor

In [ ]:
df = pd.read_csv('df_time_series.csv')
df.head()

In [ ]:
dh = DataHolder(df, 'id', 'act', 'date')

In [ ]:
%%time

# предсказание только для нод на 4 дня вперёд

gsp = GSPredictor(dh, period_type='D', pred_period=4, two_models=False, edges=False, quick_mod = True)

In [ ]:
gsp.plot_nodes_prob()

In [ ]:
gsp.plot_nodes_duration()

###### Спрогнозированные вероятности нод

In [ ]:
gsp.nodes_prob_pred

###### Спрогнозированные длительности нод

In [ ]:
gsp.nodes_duration_pred

In [ ]:
graph = gsp.get_predicted_graph()
painter = GraphvizPainter()
painter.apply(graph)
painter.show()

## X. Имитационное моделирование и what-if анализ

Модуль имитационного моделирования __`sberpm.imitation`__ позволяет симулировать процесс в as-is форме, вносить изменения в процесс и проводить what-if моделирование, а также оценивать качество симуляции по сравнению с исходным лог-файлом с помощью классов `Simulation` и `SimilarityMetric` соответственно. 

In [ ]:
from sberpm.imitation import Simulation, SimilarityMetric

Методы класса `Simulation`:
- __run__ – запуск симуляции num_traces (количество) цепочек активностей (id)
- __to_initial_state__ – возврат процесса к изначальному состоянию
- __mean_duration__ – средняя длительность выполнения активностей или переходов в процессе
- __change_node_duration__ и __change_edge_duration__ – ограничение времени выполнения конкретной активности или перехода
- __delete_node__ и __delete_edge__ – удаление ноды (активности) и ребра (перехода) из процесса соответственно
- __get_nodes_insights__ и __get_edges_insights__ – поиск инсайтов по нодам и ребрам процесса соответственно
- __delete_nodes_insights__ и __delete_edges_insights__ – удаление инсайтов, обнаруженных по нодам и ребрам процесса соответственно 

На вход класс принимает обьект типа DataHolder. Можно также зафиксировать __random_state__, чтобы результаты были воспроизводимыми. 

In [ ]:
# Инициализация
sim = Simulation(data_holder) 

### As-is моделирование

Метод __run__ запускает симуляцию процесса. Он имеет параметры __num_traces__ – число цепочек событий для симуляции и __max_trace_length__ – максимальная длина сгенерированной цепочки (по умолчанию 100). 

In [ ]:
# Симуляция исходного числа цепочек (id)
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))
sim_data.head()

Полученный лог можно отрисовать с помощью майнера и встроенного инструмента для визуализации графов.

In [ ]:
# Создание DataHodler для сгенерированных данных
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Майнер
miner = HeuMiner(holder_sim)
miner.apply()

# Отрисовка
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

### Валидация модели

Качество симуляции можно проверить с помощью класса `SimilarityMetric`, который измеряет сходство исходного и сгенерированного лога с помощью _расстояния Дамерау-Левенштейна_. На вход класс принимает лог, полученный в ходе симуляции, и DataHolder исходного файла. Результат хранится в поле __similarity__.

In [ ]:
%%time

sim_metric = SimilarityMetric(sim_data, data_holder)
print('Сходство:', sim_metric.similarity)

Можно также посмотреть метрику сходства для каждой цепочки из сгенерированного лога отдельно.

In [ ]:
sim_metric.result.head()

Качество симуляции невысокое из-за особенностей синтетического датасета. На реальном лог-файле качество значительно выше.

### What-if анализ

Методы __delete_node__ и __delete_edge__ позволяют удалить активность или ребро в процессе. После запуска симуляции процесс будет реализовываться по альтернативным путям. Параметры __node__ и __edge__ – названия активности и перехода в процессе соответственно.

In [ ]:
sim.delete_node('Stage_7')

In [ ]:
# Симуляция процесса без ноды 
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))

# Создание DataHolder
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Майнер
miner = HeuMiner(holder_sim)
miner.apply()

# Отрисовка
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

Для возврата процесса к исходному состоянию необходимо воспользоваться методом __to_initial_state__.

In [ ]:
sim.to_initial_state()

С помощью метода __mean_duration__ можно проанализировать среднюю продолжительность выполнения всех активностей или переходов в процессе (зависит от параметра  __mode__).

In [ ]:
sim.mean_duration(target='activities')

Методы __change_activity_duration__ и __change_transition_duration__ позволяют ограничить время выполнения конкретной активности или перехода. Они имеют следующие параметры:
- __activity__ или __transition__ – название активности или перехода для ограничения
- __threshold__ – максимальная продолжительность выполнения активности или перехода
- __scale__ – коэффициент уменьшения (если > 1) или увеличения (если < 1) продолжительности выполнения активности или перехода

In [ ]:
sim.change_activity_duration(activity='Stage_0', scale=3)  # уменьшить в 3 раза

In [ ]:
# Симуляция с уменьшенной длительностью ноды
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))

# Создание DataHolder
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Расчет метрик для добавления на граф
time_metric = ActivityMetric(holder_sim, time_unit='s')
mean_time_node_metric = time_metric.mean_duration().fillna(0).to_dict()

# Майнер
miner = HeuMiner(holder_sim)
miner.apply()
graph = miner.graph
graph.add_node_metric('mean_time', mean_time_node_metric)

# Отрисовка
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

In [ ]:
sim.mean_duration('activities')

In [ ]:
# Возврат к изначальному состоянию
sim.to_initial_state()

В рамках what-if анализа можно также произвести поиск инсайтов с помощью метода __get_insights__, основанных на модуле автоматического поиска инсайтов, и удалить их с помощью __delete_activities_insights__ и __delete_transitions_insights__ соответственно. 

In [ ]:
sim.get_insights(mode='activities')

In [ ]:
sim.delete_activities_insights()

In [ ]:
# Симуляция процесса без инсайтов 
sim_data = sim.run(num_of_traces=len(data_holder.grouped_data))

# Создание DataHolder
holder_sim = DataHolder(sim_data, 'id', 'stages', 'dt')

# Майнер
miner = HeuMiner(holder_sim)
miner.apply()

# Отрисовка
painter = GraphvizPainter()
painter.apply(miner.graph)
painter.show()

##  XI. Decision Mining

Модуль __`sberpm.decision_mining`__ предназначен для проведения __decision point analysis__, который заключается в определении причин, почему процесс идет по тому или иному пути. Класс `DecisionMining` выявляет, как те или иные свойства (атрибуты) процесса влияют на выбор конкретного пути. 

In [ ]:
from sberpm.decision_mining import DecisionMining

На вход `DecisionMining` принимает объект типа DataHolder. 

In [ ]:
# Инициализация
dm = DecisionMining(data_holder)

`DecisionMining` имеет следующие методы:
- __print_decision_points__ – выводит decision points (активности, после которых идет выбор)
- __apply__ – выполняет анализ decision points, строя дерево решений по указанным атрибутам
- __get_clf_metrics__ – выводит метрики классификации
- __plot_confusion_matrix__ – рисует матрицы ошибок
- __plot_feature_importance__ – рисует важность признаков в дереве
- __plot_feature_distribution__ – рисует распределение признаков по классам 
- __plot_decision_tree__ – рисует дерево решений
- __print_decision_rule__ – выводит решающие правила

Decision points – точки, где процесс имеет разветвление, можно посмотреть с помощью метода __print_decision_points__.

In [ ]:
dm.print_decision_points()

Метод __apply__ запускает алгоритм decision mining. Он имеет следующие параметры:
- __categorical_attrs__ – названия категориальных признаков
- __noncategorical_attrs__ – названия некатегориальных признаков
- __decision_points__ – точки, по которым необходимо построить деревья решений, по умолчанию рассматриваются все
- __sampling__ – нужен ли sampling (over- или under-), следует использовать в случае несбалансированных классов
- __tree_params__ – параметры дерева решений
- __grid_search__ – нужен ли подбор оптимальных гиперпараметров дерева решений
- __param_grid__ – сетка параметров, используется только при grid_search=True
- __random_state__ – используется в дереве решений и sampling
- __n_jobs__ – используется в sampling и grid_search

In [ ]:
dm.apply(categorical_attrs=[data_holder.user_column],
         noncategorical_attrs=[data_holder.duration_column],
         decision_points='all', 
         sampling='RandomOverSampler',
         tree_params='default',
         grid_search=False, 
         param_grid='default',
         random_state=42,
         n_jobs=None)

Результаты классификации можно посмотреть с помощью методов __get_clf_metrics__, __plot_confusion_matrix__ и __plot_feature_importance__. 

In [ ]:
dm.get_clf_metrics()

Качество невысокое из-за особенностей синтетического датасета.

Все plot методы имеют параметры:
- __decision_points__ – точки, для которых необходимо изобразить харатеристики
- __savefig__ – нужно ли сохранить изображение

In [ ]:
dm.plot_confusion_matrix(decision_points=['Stage_0'], savefig=False)

In [ ]:
dm.plot_feature_importance(decision_points=['Stage_0'], savefig=False)

Метод __plot_feature_importance__ дополнительно имеет еще два параметра:
- __drop_outliers__ – нужно ли удалить выбросы для количественных признаков
- __clf_results__ – нарисовать распределение признаков по результатам классификации (True) или по исходному логу (False)

In [ ]:
dm.plot_feature_distribution(decision_points=['Stage_0'], drop_outliers=True, clf_results=True, savefig=False)

Методы __plot_decision_tree__ и __print_decision_rule__ выводят результаты работы decision mining алгоритма в виде дерева и правил соответственно.

Параметры __plot_decision_tree__:
- __decision_points__ – точки, для которых необходимо нарисовать дерево решений
- __max_depth__ – максимальная глубина дерева
- __scale__ – масштаб графика
- __savefig__ – нужно ли сохранить изображение

In [ ]:
dm.plot_decision_tree(decision_points=['Stage_0'], max_depth=None, scale=1, savefig=False)

Параметры __print_decision_rule__:
- __decision_points__ – точки, для которых необходимо вывести решающие правила
- __paths__ – пути, по которым необходимо вывести решающие правила

In [ ]:
dm.print_decision_rule(decision_points=['Stage_0'], paths=['Stage_1'])

## XII. Хронометраж

Расчёт длительности процесса с предварительной очисткой вывбросов при помоощи алгоритмов машинного обучения.

- __data_holder (SberPM DataHolder)__ - класс с хранящимися данными 
- __start_query__ (str) - запрос указывающий на начало нового процесса
- __end_query__ (str) - запрос указывающий на окончание процесса
- __query__ (str) - запрос указывающий на начало нового процесса или на завершение процесса в данной строке
- __change_columns__ (List[str]) - список с названиями колонок по которым можно определить что начался новый процесс при изменении значения в колонке (например изменения идентификатора процесса или пользователя) 
- __sort_params__ (List[str]) - список названий колонок по которым будет производиться предварительная сортировка данных

Параметры __query__, __start_query__, __end_query__ могут быть типа __"sql"__ или __"pandas"__, они оба должны ссылаться на фрейм данных как __"df"__, они должны возвращать один столбец: булевую маску или столбец из 0 и 1.

В случае, если задаётся __"sql"__ запрос, то он должен выглядеть как __"SELECT ... from df"__.

Метод __get_chrono()__ начнёт процесс расчёта длительности процесса и в результате выведет словарь(dict) с элементами:
- среднее временя процесса в секундах
- количество отобранных элементов
- количество уникальных процессов
- максимальное количество уникальных идентификаторов рассчитанных в хронометраже 


In [ ]:
from sberpm.ml.chronometrage import Chronometrage

In [ ]:
df = pd.read_excel('chrono_data.xlsx', engine='openpyxl')
dh = DataHolder(df, 'process_id', 'event_type', 'data_timestamp')

In [ ]:
example_start_query = """(df['event_type'] == 'Процесс_16961') & (df['event_action'].isin(['Начало']))"""
example_end_query = """(df['event_type'] == 'Процесс_16961') & (df['event_action'].isin(['Конец']))"""

In [ ]:
cr = Chronometrage(dh, 
                   sort_params=['process_id', 'user_id', 'data_timestamp'], 
                   start_query=example_start_query,
                   end_query=example_end_query,
                   change_columns=['process_id', 'user_id'])
res = cr.get_chrono()

In [ ]:
res